In [1]:
import numpy as np
import pandas as pd
import random
from matplotlib import pyplot as plt
from numba import jit
from tqdm import tqdm
from itertools import combinations, permutations
import time

In [2]:
data_A = np.loadtxt('../data/TSPA.csv', delimiter=';').astype(np.int64)
data_B = np.loadtxt('../data/TSPB.csv', delimiter=';').astype(np.int64)
data_C = np.loadtxt('../data/TSPC.csv', delimiter=';').astype(np.int64)
data_D = np.loadtxt('../data/TSPD.csv', delimiter=';').astype(np.int64)

In [3]:
def create_cost_matrix(data):
    x = data[:, :1]
    y = data[:, 1:2]
    cost = data[:, 2:3]
    return (((x - x.reshape(1, -1))**2 + (y - y.reshape(1, -1))**2) ** (1/2) + cost.reshape(1, -1)).round().astype(np.int64)

def create_dist_matrix(data):
    x = data[:, :1]
    y = data[:, 1:2]
    #cost = data[:, 2:3]
    return (((x - x.reshape(1, -1))**2 + (y - y.reshape(1, -1))**2) ** (1/2)).round().astype(np.int64)

In [4]:
cost_matrix_A = create_cost_matrix(data_A)

In [5]:
cost_matrix_A

array([[  84, 2032, 2098, ..., 4159, 3783, 1514],
       [1633,  483, 2398, ..., 3349, 2266,  817],
       [ 720, 1419, 1462, ..., 3640, 3149,  964],
       ...,
       [2782, 2371, 3641, ..., 1461, 2908, 2554],
       [2558, 1440, 3302, ..., 3060, 1309, 1773],
       [1234,  936, 2062, ..., 3651, 2718,  364]], dtype=int64)

In [6]:
cost_matrix_B = create_cost_matrix(data_B)

In [7]:
cost_matrix_C = create_cost_matrix(data_C)

In [8]:
cost_matrix_D = create_cost_matrix(data_D)

In [9]:
dist_matrix_A = create_dist_matrix(data_A)
dist_matrix_B = create_dist_matrix(data_B)
dist_matrix_C = create_dist_matrix(data_C)
dist_matrix_D = create_dist_matrix(data_D)

In [10]:
def plot(data, solution):
    data_ordered = np.array([data[i] for i in solution])
    all_data = np.array([data[i] for i in range(200)])

    plt.figure(figsize=(10, 10), dpi=80)

    plt.scatter(data_ordered[:,0], data_ordered[:,1], s=data_ordered[:,2]/data_ordered[:,2].max()*200, c='b')
    plt.scatter(all_data[:,0], all_data[:,1], s=all_data[:,2]/all_data[:,2].max()*200, c='b')
    plt.plot(data_ordered[:,0], data_ordered[:,1], 'y-')
    plt.plot([data_ordered[0,0], data_ordered[-1,0]], [data_ordered[0,1], data_ordered[-1,1]], 'y-')
    plt.show()

In [11]:
def calculate_performance(cycle, cost_matrix):
    total_sum = 0
    for i in range(len(cycle)-1):
        total_sum += cost_matrix[cycle[i], cycle[i+1]]
    total_sum += cost_matrix[cycle[-1], cycle[0]]
    return total_sum

In [12]:
@jit()
def random_solution(cost_matrix, limit=100):
    random_solution_list = list(range(0,len(cost_matrix)))
    random.shuffle(random_solution_list)
    return np.array(random_solution_list)[:limit]

solution = random_solution(cost_matrix_A, 100)
print(solution)

[143 124 192  16  66 117  21 126 123  17   0  65  57 176  97 142 180 196
 193  15 161 131  94 155 137  76 103   1 181  84  63  25  77 177 125 102
  32  40  34  83  86  73 118 156  41  47  38 150 190  91  87 147  51  36
  14  71 141  81 166 121 198 113  75 179 194 104 146 154  90 112  80  37
  18  39 168  52 138 145 110 186  59 183 116  43   7 159  20 134  89  26
  33  93  29  13 151  42 184 191 149  28]


C:\Users\hubra\AppData\Local\Temp\ipykernel_496\3945026401.py:1: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "random_solution" failed type inference due to: No implementation of function Function(<bound method Random.shuffle of <random.Random object at 0x0000016357929D50>>) found for signature:
 
 >>> shuffle(list(int64)<iv=None>)
 
There are 2 candidate implementations:
  - Of which 2 did not match due to:
  Overload in function 'shuffle_impl': File: numba\cpython\randomimpl.py: Line 1230.
    With argument(s): '(list(int64)<iv=None>)':
   Rejected as the implementation raised a specific error:
     TypeError: The argument to shuffle() should be a buffer type
  raised from C:\Users\hubra\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\numba\cpython\randomimpl.py:1206

During: resolving callee type: Function(<bound method Random.shuffle of <random.Random o

In [13]:
@jit()
def greedy_cycle(cost_matrix, current_id, limit=100):
    all_ids = set(list(range(0,len(cost_matrix))))
    all_ids.remove(current_id)
    solution = [current_id]
    
    for _ in range(1):
        min_val = 99999
        min_id = -1
        for next_id in all_ids:
            if cost_matrix[current_id][next_id] < min_val:
                min_val = cost_matrix[current_id][next_id]
                min_id = next_id
        solution.append(min_id)
        all_ids.remove(min_id)
        current_id = min_id
    
    while len(solution) < limit:
        min_delta = 99999
        min_id = -1
        insert_id = -1
        for i in range(len(solution)-1):
            for next_id in all_ids:
                delta = cost_matrix[solution[i]][next_id] + cost_matrix[next_id][solution[i+1]] - cost_matrix[solution[i]][solution[i+1]]
                if delta < min_delta:
                    min_delta = delta
                    min_id = next_id
                    insert_id = i
        for next_id in all_ids:
            delta = cost_matrix[solution[-1]][next_id] + cost_matrix[next_id][solution[0]] - cost_matrix[solution[-1]][solution[0]]
            if delta < min_delta:
                min_delta = delta
                min_id = next_id
                insert_id = i
        solution.insert(insert_id+1, min_id)
        all_ids.remove(min_id)

    return np.array(solution)

In [14]:
# swap node in cycle and unused node
def inter_exchange_delta(cycle, cost_matrix, used_cycle_id, unused_node):
    if used_cycle_id == len(cycle)-1:
        return cost_matrix[cycle[used_cycle_id-1], unused_node] + cost_matrix[unused_node, cycle[0]] - cost_matrix[cycle[used_cycle_id-1], cycle[used_cycle_id]] - cost_matrix[cycle[used_cycle_id], cycle[0]]
    return cost_matrix[cycle[used_cycle_id-1], unused_node] + cost_matrix[unused_node, cycle[used_cycle_id+1]] - cost_matrix[cycle[used_cycle_id-1], cycle[used_cycle_id]] - cost_matrix[cycle[used_cycle_id], cycle[used_cycle_id+1]]

In [15]:
def intra_two_nodes_exchange_delta(cycle, cost_matrix, node1, node2, pri = False):
    # print(node1, node2)
    if node1 == len(cycle)-1:
        node1plus = 0
    else:
        node1plus = node1 + 1
    
    if node2 == len(cycle)-1:
        node2plus = 0
    else:
        node2plus = node2 + 1

    if abs(node1-node2) == 1:
        if node1 > node2:
            node1, node2 = node2, node1
            node1plus, node2plus = node2plus, node1plus
        return cost_matrix[cycle[node1-1], cycle[node2]] + cost_matrix[cycle[node2], cycle[node1]] + cost_matrix[cycle[node1], cycle[node2plus]] - cost_matrix[cycle[node1-1], cycle[node1]] - cost_matrix[cycle[node1], cycle[node2]] - cost_matrix[cycle[node2], cycle[node2plus]]
    if abs(node1-node2) == len(cycle)-1:
        if node1 < node2:
            node1, node2 = node2, node1
            node1plus, node2plus = node2plus, node1plus
        return cost_matrix[cycle[node1-1], cycle[node2]] + cost_matrix[cycle[node2], cycle[node1]] + cost_matrix[cycle[node1], cycle[node2plus]] - cost_matrix[cycle[node1-1], cycle[node1]] - cost_matrix[cycle[node1], cycle[node2]] - cost_matrix[cycle[node2], cycle[node2plus]]

    return cost_matrix[cycle[node1-1], cycle[node2]] + cost_matrix[cycle[node2], cycle[node1plus]] + cost_matrix[cycle[node2-1], cycle[node1]] + cost_matrix[cycle[node1], cycle[node2plus]] - cost_matrix[cycle[node1-1], cycle[node1]] - cost_matrix[cycle[node1], cycle[node1plus]] - cost_matrix[cycle[node2-1], cycle[node2]] - cost_matrix[cycle[node2], cycle[node2plus]]

In [16]:
def intra_two_edges_exchange_delta(cycle, dist_matrix, node1, node2):
    if node1 == len(cycle)-1 or node2 == len(cycle)-1:
        return 99999
    #print(cost_matrix[cycle[node1], cycle[node2]], cost_matrix[cycle[node1+1], cycle[node2+1]], cost_matrix[cycle[node1], cycle[node1+1]], cost_matrix[cycle[node2], cycle[node2+1]])
    return dist_matrix[cycle[node1], cycle[node2]] + dist_matrix[cycle[node1+1], cycle[node2+1]] - dist_matrix[cycle[node1], cycle[node1+1]] - dist_matrix[cycle[node2], cycle[node2+1]]


In [55]:
jit()
def steepest_local_search(cost_matrix, dist_matrix, inter = True, initial_greedy = False, starting_node = None):
    if initial_greedy:
        if starting_node:
            cycle = greedy_cycle(cost_matrix, starting_node)
        else:
            cycle = greedy_cycle(cost_matrix, random.randint(0, len(cost_matrix)-1))
    else:
        cycle = random_solution(cost_matrix)
    unused_nodes = set(list(range(0,len(cost_matrix))))
    for node in cycle:
        unused_nodes.remove(node)

    all_combinations = list(combinations(list(range(0, 100)), 2))

    while(True):
        best_delta, swap_node_a, swap_node_b, best_type = 0, None, None, None
        if inter:
            for used_cycle_id in range(len(cycle)):
                for unused_node in unused_nodes:
                    delta = inter_exchange_delta(cycle, cost_matrix, used_cycle_id, unused_node)
                    if delta < best_delta:
                        best_delta, swap_node_a, swap_node_b, best_type = delta, used_cycle_id, unused_node, 'inter_exchange'
        else:
            for x, y in all_combinations:
                delta = intra_two_nodes_exchange_delta(cycle, cost_matrix, x, y)
                if delta < best_delta:
                    best_delta, swap_node_a, swap_node_b, best_type = delta, x, y, 'intra_two_nodes_exchange'
            for x, y in all_combinations:
                delta = intra_two_edges_exchange_delta(cycle, dist_matrix, x, y)
                if delta < best_delta:
                    best_delta, swap_node_a, swap_node_b, best_type = delta, x, y, 'intra_two_edges_exchange'

        if best_type is not None:
            if best_type == 'inter_exchange':
                unused_nodes.add(cycle[swap_node_a])
                unused_nodes.remove(swap_node_b)
                cycle[swap_node_a] = swap_node_b
            elif best_type == 'intra_two_nodes_exchange':
                cycle[swap_node_a], cycle[swap_node_b] = cycle[swap_node_b], cycle[swap_node_a]
            elif best_type == 'intra_two_edges_exchange':
                cycle[swap_node_a+1], cycle[swap_node_b] = cycle[swap_node_b], cycle[swap_node_a+1]
                cycle[swap_node_a+2:swap_node_b] = cycle[swap_node_a+2:swap_node_b][::-1]
        else:
            break
    return cycle

In [56]:
jit()
def greedy_local_search(cost_matrix, dist_matrix, inter = True, initial_greedy = False, starting_node = None):
    if initial_greedy:
        if starting_node:
            cycle = greedy_cycle(cost_matrix, starting_node)
        else:
            cycle = greedy_cycle(cost_matrix, random.randint(0, len(cost_matrix)-1))
    else:
        cycle = random_solution(cost_matrix)
    unused_nodes = set(list(range(0,len(cost_matrix))))
    for node in cycle:
        unused_nodes.remove(node)

    if inter:
        neighbourhood = [(x, y) for x in range(len(cycle)) for y in unused_nodes]
    else:
        neighbourhood = [(x, y, 'intra_two_nodes_exchange') for x, y in combinations(list(range(0, 100)), 2)]
        neighbourhood += [(x, y, 'intra_two_edges_exchange') for x, y in combinations(list(range(0, 100)), 2)]

    while(True):
        best_delta, swap_node_a, swap_node_b, best_type = 0, None, None, None
        random.shuffle(neighbourhood)
        if inter:
            for used_cycle_id, unused_node in neighbourhood:
                delta = inter_exchange_delta(cycle, cost_matrix, used_cycle_id, unused_node)
                if delta < best_delta:
                    best_delta, swap_node_a, swap_node_b, best_type = delta, used_cycle_id, unused_node, 'inter_exchange'
                    break
        else:
            for x, y, z in neighbourhood:
                if z == 'intra_two_nodes_exchange':
                    delta = intra_two_nodes_exchange_delta(cycle, cost_matrix, x, y)
                    if delta < best_delta:
                        best_delta, swap_node_a, swap_node_b, best_type = delta, x, y, 'intra_two_nodes_exchange'
                        break
                elif z == 'intra_two_edges_exchange':
                    delta = intra_two_edges_exchange_delta(cycle, dist_matrix, x, y)
                    if delta < best_delta:
                        best_delta, swap_node_a, swap_node_b, best_type = delta, x, y, 'intra_two_edges_exchange'
                        break

        if best_type is not None:
            if best_type == 'inter_exchange':
                unused_nodes.add(cycle[swap_node_a])
                unused_nodes.remove(swap_node_b)
                cycle[swap_node_a] = swap_node_b
                neighbourhood = [(x, y) for x in range(len(cycle)) for y in unused_nodes]
            elif best_type == 'intra_two_nodes_exchange':
                cycle[swap_node_a], cycle[swap_node_b] = cycle[swap_node_b], cycle[swap_node_a]
            elif best_type == 'intra_two_edges_exchange':
                cycle[swap_node_a+1], cycle[swap_node_b] = cycle[swap_node_b], cycle[swap_node_a+1]
                cycle[swap_node_a+2:swap_node_b] = cycle[swap_node_a+2:swap_node_b][::-1]
        else:
            break
    return cycle

In [57]:
steepest_local_search(cost_matrix_A, dist_matrix_A, inter = True, initial_greedy = False)
greedy_local_search(cost_matrix_A, dist_matrix_A, inter = True, initial_greedy = False)

array([128, 186,  72,  80,   8,  48,  26, 101, 167,  45,  21, 194, 157,
        76, 145,  91,  69,  86,  43, 178, 147,  59,  27,  20,  64,  71,
       163, 180,  32,  79, 133,  81, 102, 141,   6, 112,  51, 153,  88,
        55, 195,  62, 154,  87, 144, 161, 164,  96, 185, 132,  22, 117,
       108,  53, 159, 143,  19,  50, 175, 114,  75, 119, 189, 121, 149,
       115,  35,   0, 113, 181,  74, 171,  98,  73,  31,  94,  12,  66,
        68, 190,  24, 156, 172, 127,   4, 177,   1, 109, 134, 135,  95,
       169, 139, 130, 152,  11, 199, 192,  77,  36])

Tests

In [20]:
def present_results(min, max, average, time):
    print("{: >10} {: >10} {: >10} {: >10}".format('MIN', 'MAX', 'AVG', 'TIME'))
    print("{: >10} {: >10} {: >10} {: >10}".format(min, max, average, time))
    print()

In [54]:
# local_search(cost_matrix, inter = True, initial_greedy = False, starting_node = None)
def test_solution(local_search, cost_matrix, dist_matrix, inter, initial_greedy):
    start = time.time()
    costs = []
    for i in range(len(cost_matrix)):
        cycle = local_search(cost_matrix, dist_matrix, inter, initial_greedy, i)
        total_cost = calculate_performance(cycle, cost_matrix)
        costs.append(total_cost)
    costs = np.array(costs)
    return costs.min(), costs.max(), costs.mean(), time.time() - start

local search | neighborhood | starting solutions

In [59]:
for i in range(2,4):
    if i == 0:
        cost_matrix = cost_matrix_A
        dist_matrix = dist_matrix_A
        print('DATASET A')
    elif i == 1:
        cost_matrix = cost_matrix_B
        dist_matrix = dist_matrix_B
        print('DATASET B')
    elif i == 2:
        cost_matrix = cost_matrix_C
        dist_matrix = dist_matrix_C
        print('DATASET C')
    elif i == 3:
        cost_matrix = cost_matrix_D
        dist_matrix = dist_matrix_D
        print('DATASET D')

    print('STEEPEST | INTER | GREEDY')
    present_results(*test_solution(steepest_local_search, cost_matrix, dist_matrix, True, True))
    print('STEEPEST | INTER | RANDOM')
    present_results(*test_solution(steepest_local_search, cost_matrix, dist_matrix, True, False))
    print('STEEPEST | INTRA | GREEDY')
    present_results(*test_solution(steepest_local_search, cost_matrix, dist_matrix, False, True))
    print('STEEPEST | INTRA | RANDOM')
    present_results(*test_solution(steepest_local_search, cost_matrix, dist_matrix, False, False))
    print('GREEDY | INTER | GREEDY')
    present_results(*test_solution(greedy_local_search, cost_matrix, dist_matrix, True, True))
    print('GREEDY | INTER | RANDOM')
    present_results(*test_solution(greedy_local_search, cost_matrix, dist_matrix, True, False))
    print('GREEDY | INTRA | GREEDY')
    present_results(*test_solution(greedy_local_search, cost_matrix, dist_matrix, False, True))
    print('GREEDY | INTRA | RANDOM')
    present_results(*test_solution(greedy_local_search, cost_matrix, dist_matrix, False, False))
    print('===================================')

DATASET C
STEEPEST | INTER | GREEDY
       MIN        MAX        AVG       TIME
     53226      58806    55914.9 5.875683307647705

STEEPEST | INTER | RANDOM
       MIN        MAX        AVG       TIME
     67412      87349  77047.685 263.54118633270264

STEEPEST | INTRA | GREEDY
       MIN        MAX        AVG       TIME
     53080      58611   55391.54 39.83351492881775

STEEPEST | INTRA | RANDOM
       MIN        MAX        AVG       TIME
     69568      80370   74423.61 430.69493412971497

GREEDY | INTER | GREEDY
       MIN        MAX        AVG       TIME
     52956      58937   55909.44 8.243127822875977

GREEDY | INTER | RANDOM
       MIN        MAX        AVG       TIME
     65079      84722  73641.395 383.6931369304657

GREEDY | INTRA | GREEDY
       MIN        MAX        AVG       TIME
     53080      58611   55415.78 23.01422429084778

GREEDY | INTRA | RANDOM
       MIN        MAX        AVG       TIME
     68635      80714  74783.725 322.87251496315

DATASET D
STEEPEST | I